In [8]:
 import sys
#Second append the folder path

sys.path.insert(0, './../inc')
sys.path.insert(0, './../src')

from classfier_tools import plot_confusion_matrix, classic_classifier
from pdal_pipline import las_add_features, las_2_dataframe

In [9]:
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pyntcloud import PyntCloud
from pyntcloud import scalar_fields
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors.kde import KernelDensity
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pickle

#import pclpy
# install this pip sudo apt install libgl1-mesa-glx
import open3d as o3d

import json
import pdal

#from pointcloud import cloud_features, cloud_to_featurs_datasets

#import keras
#import tensorflow

import ipyleaflet
import IPython.display
import ipyvolume.pylab as p3
import json

import numpy as np
x = np.random.rand(100, 3)
import pptk
v = pptk.viewer(x)
v.set(point_size=0.01)

In [10]:
classe_labels = {
    "noise/other"  : 7,
    "ground"       : 2,
    "road"         : 11,
    "vegetation"   : 3,
    "building"     : 6,
    "light"        : 19,
    "signal"       : 20,
    "vehicle"      : 21,
    "fence/people" : 22
    }

classe_labels

{'noise/other': 7,
 'ground': 2,
 'road': 11,
 'vegetation': 3,
 'building': 6,
 'light': 19,
 'signal': 20,
 'vehicle': 21,
 'fence/people': 22}

In [11]:
from pathlib import Path
extension = '.bpf'
files=[]
PrjPath = Path('./../../')
i = 0
for entry in PrjPath.rglob(f'*{extension}'): # PrjPath.iterdir():
    print(i, entry)
    files.append(entry)
    i+=1

#print(files)
#LIDAR_FILENAME = files[16].resolve()
#print(LIDAR_FILENAME)

extension = '.shp'
shp_files=[]

i = 0
for entry in PrjPath.rglob(f'*{extension}'): # PrjPath.iterdir():
    print(i, entry)
    shp_files.append(entry)
    i+=1
    
#print(files)
#LIDAR_FILENAME = files[16].resolve()
#print(LIDAR_FILENAME)

extension = '.las'
LAS_files=[]

i = 0
for entry in PrjPath.rglob(f'*{extension}'): # PrjPath.iterdir():
    print(i, entry)
    LAS_files.append(entry)
    i+=1

0 ../../Werk2_part1_features.bpf
1 ../../Werk2_part2_features.bpf
0 ../../Werk2_part1_features.shp
1 ../../shape/086b_classified.shp
0 ../../Werk2_part1_features-Copy2.las
1 ../../Werk2_part1_features.las
2 ../../Werk2_part1_features-Copy1.las


In [12]:
las_file = str(LAS_files[0])

In [13]:
lidar_df = las_2_dataframe(las_file)
lidar_df.head()

True
Time taken  1.8443849086761475  seconds
Number of points in LiDAR: 2044148


,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,ScanAngleRank,...,Coplanar,HeightAboveGround,Linearity,Planarity,Scattering,Omnivariance,Anisotropy,Eigentropy,Eigen_Sum,Curvature_Change
0,535030.20,5278937.90,455.43,0,1,1,0,0,2,0.0,...,1,0.00,0.948911,0.051089,0.0,0.0,1.0,NaN,0.001794,0.0
1,535026.84,5278936.73,455.77,0,1,1,0,0,1,0.0,...,1,0.51,0.949710,0.050290,0.0,0.0,1.0,NaN,0.001908,0.0
2,535026.40,5278935.53,455.73,0,1,1,0,0,1,0.0,...,0,0.33,0.947047,0.052953,0.0,0.0,1.0,NaN,0.001913,0.0
3,535026.47,5278936.56,455.73,0,1,1,0,0,2,0.0,...,1,0.00,0.943249,0.056751,0.0,0.0,1.0,NaN,0.001919,0.0
4,535026.50,5278935.51,455.72,0,1,1,0,0,2,0.0,...,1,0.00,0.949436,0.050564,0.0,0.0,1.0,NaN,0.002142,0.0


In [16]:
ground_df = lidar_df[lidar_df['Classification']!=2]
print(ground_df.shape)
ground_df

(332534, 39)


,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,ScanAngleRank,...,Coplanar,HeightAboveGround,Linearity,Planarity,Scattering,Omnivariance,Anisotropy,Eigentropy,Eigen_Sum,Curvature_Change
1,535026.84,5278936.73,455.77,0,1,1,0,0,1,0.0,...,1,0.51,0.949710,0.050290,0.000000,0.000000,1.000000,NaN,0.001908,0.000000
2,535026.40,5278935.53,455.73,0,1,1,0,0,1,0.0,...,0,0.33,0.947047,0.052953,0.000000,0.000000,1.000000,NaN,0.001913,0.000000
5,535025.94,5278935.47,455.74,0,1,1,0,0,1,0.0,...,0,0.35,0.948669,0.051331,0.000000,0.000000,1.000000,NaN,0.002181,0.000000
9,535026.75,5278935.52,455.78,0,1,1,0,0,1,0.0,...,0,0.35,0.949473,0.050527,0.000000,0.000000,1.000000,NaN,0.002320,0.000000
10,535025.79,5278935.46,455.77,0,1,1,0,0,1,0.0,...,0,0.10,0.949161,0.050839,0.000000,0.000000,1.000000,NaN,0.002321,0.000000
11,535025.70,5278935.44,455.77,0,1,1,0,0,1,0.0,...,0,0.44,0.949579,0.050421,0.000000,0.000000,1.000000,NaN,0.002369,0.000000
12,535026.22,5278936.47,455.75,0,1,1,0,0,1,0.0,...,0,0.39,0.949579,0.050421,0.000000,0.000000,1.000000,NaN,0.002369,0.000000
13,535026.23,5278935.49,455.76,0,1,1,0,0,1,0.0,...,0,0.27,0.946979,0.053021,0.000000,0.000000,1.000000,NaN,0.002380,0.000000
14,535024.68,5278935.31,455.77,0,1,1,0,0,1,0.0,...,0,0.52,0.946703,0.053297,0.000000,0.000000,1.000000,NaN,0.002380,0.000000
15,535025.32,5278935.40,455.74,0,1,1,0,0,1,0.0,...,0,0.26,0.945122,0.054878,0.000000,0.000000,1.000000,NaN,0.002384,0.000000


In [15]:
fig = p3.figure(width=1000)
fig.xlabel='Y'
fig.ylabel='Z'
fig.zlabel='X'
all_points = p3.scatter(ground_df['Y'], ground_df['Z'], ground_df['X'], color='red', size=.2)
p3.squarelim()
p3.show()

KeyError: 0

In [ ]:
df = lidar_df
df['X_0'] = df['X']
df['Y_0'] = df['Y']
df['Z_0'] = df['Z']
df['X'] = df['X'] - df['X_0'].min()
df['Y'] = df['Y'] - df['Y_0'].min()
df['Z'] = df['Z'] - df['Z_0'].min()

In [ ]:
fig = p3.figure(width=1000)
fig.xlabel='Y'
fig.ylabel='Z'
fig.zlabel='X'
all_points = p3.scatter(df['Y'], df['Z'], df['X'], color='red', size=.2)
p3.squarelim()
p3.show()

In [ ]:
# Color ground in grey
df['ground'] = df['Classification']==2
ground = p3.scatter(df.loc[df['ground'],'Y'], df.loc[df['ground'],'Z'], df.loc[df['ground'],'X'], color='red', size=.2)
non_ground = p3.scatter(df.loc[~df['ground'],'Y'], df.loc[~df['ground'],'Z'], df.loc[~df['ground'],'X'], color='red', size=.2)
fig.scatters.append(ground)
fig.scatters.append(non_ground)
all_points.visible = False
ground.color='lightgrey'

In [ ]:
df['ground']

In [ ]:
(lidar_df[["Eigen_Sum", "Eigenvalue0", "Eigenvalue1", "Eigenvalue2", "Curvature_Change", 'Curvature', 'RadialDensity',
       'Coplanar', 'HeightAboveGround', 'Linearity', 'Planarity', 'Scattering',
       'Omnivariance', 'Anisotropy', 'Eigentropy', 'Eigen_Sum',]].head(100))

In [ ]:
np.max(lidar_df[ "Eigenvalue0"])

In [ ]:
print(lidar_df[ "Eigenvalue0"] + lidar_df["Eigenvalue1"]  + lidar_df["Eigenvalue2"])

In [ ]:
len( lidar_df.columns)

In [ ]:
lidar_df.columns

In [ ]:
 KD-tree

In [ ]:
a = np.array([[1,4,2], [3,1,6], [12, 11, 16], [12, 11, 16], [12, 17, 12], [12, 16, 13]])

In [ ]:
np.sort(a, axis=1)

In [ ]:
print(a.sort(a))

In [ ]:
np.sum(a, axis=1)

In [ ]:
a[:,1] + a[:,2]

In [ ]:
    evalue1 = a[:,2] # largest
    evalue2 = a[:,1]
    evalue3 = a[:,0] # smallest
    print(evalue1)
    print(evalue2)
    print(evalue3)
    # Linearity
    lambda1 = (evalue1 - evalue2) / evalue1
    print(lambda1)
    # Planarity
    lambda2 = (evalue2 - evalue3) / evalue1
    print(lambda2)

In [ ]:
np.divide(np.subtract(evalue1, evalue2), evalue1) 

In [ ]:
lambda4 = pow(evalue1*evalue2*evalue3, 1/3.0)
lambda4

In [ ]:
    s = 0
    for elem in a[:,]:
        print(elem)
        s = s + (elem*np.log(elem))
    lambda6 = (-1)*s 

In [ ]:
lambda6

In [ ]:
a

In [ ]:
def replaceZeroes(data):
  min_nonzero = np.min(data[np.nonzero(data)])
  data[data == 0] = min_nonzero
  return data


prob = replaceZeroes(prob)
result = numpy.where(prob > 0.0000000001, numpy.log10(prob), -10)

In [ ]:
-np.sum(np.multiply(a, np.log(a)), axis = 1)

In [ ]:
np.multiply(a, np.log(a))

In [ ]:
a.T.shape

In [ ]:
a.T

In [ ]:
np.array(lidar_df[ "Eigenvalue0"])

In [ ]:
D = np.array([lidar_df[ "Eigenvalue0"], lidar_df["Eigenvalue1"], lidar_df["Eigenvalue2"]])
D

In [ ]:
D[:,1]

In [ ]:
D.shape
D.T
D

In [ ]:
D.sort(axis=0)
D

In [ ]:
# np.sum(D, axis=1)
print(D)
# Get eigenvectors
# e1 = V[2] # eigenvector in direction of largest variance
# e2 = V[1] # second eigenvector, perpend. to e1
# e3 = V[0]
# Find the eigenvalues
evalue1 = D[:,2] # largest
evalue2 = D[:,1]
evalue3 = D[:,0] # smallest
print(evalue1, "\n\n\n", evalue2, "\n\n\n", evalue3)

In [ ]:
np.min(D)

In [ ]:
lambda6 = -np.sum(np.multiply(D, np.log(D)), axis = 0)

In [ ]:
evalue1

In [ ]:
np.multiply(evalue1[:,].T, np.log(evalue1[:,].T))

In [ ]:
-np.sum(np.multiply(evalue1, np.log(evalue1)), axis = 0)

In [ ]:
(-np.sum(np.multiply(D[:,], np.log(D[:,])), axis = 0)).shape

In [ ]:
np.multiply(D[:,], np.log(D[:,]))

In [ ]:
(D * np.log(D)).T

In [ ]:
np.multiply(D[:,0], np.log(D[:,0]))

In [ ]:
np.multiply(D[:,1], np.log(D[:,1]))

In [ ]:
np.multiply(D[:,2], np.log(D[:,2]))

In [ ]:
lambda7 = np.sum(D, axis=1)
lambda7

In [ ]:
lambda7 = np.sum(D, axis=0)
lambda7